In [1]:
import ffn 
from empyrical import alpha_beta
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import discrete_allocation
import matplotlib as pyplot
import numpy as np
import pandas as pd

## Getting stock prices and rebasing to view prices on common scales

In [6]:
prices = ffn.get('msft,aapl,amzn,fb,brk-b,jnj', start='2016-01-01')
benchmark = ffn.get('spy', start='2016-01-01')
msft = ffn.get('msft', start='2016-01-01')
ax = prices.rebase().plot()

print('Type of prices:', type(prices))

Type of prices: <class 'pandas.core.frame.DataFrame'>


In [4]:
returns = prices.to_returns().dropna()
ax = returns.hist(figsize=(10,10))

In [8]:
stats = prices.calc_stats()
stats.display()

Stat                 msft        aapl        amzn        fb          brkb        jnj
-------------------  ----------  ----------  ----------  ----------  ----------  ----------
Start                2016-01-04  2016-01-04  2016-01-04  2016-01-04  2016-01-04  2016-01-04
End                  2020-12-31  2020-12-31  2020-12-31  2020-12-31  2020-12-31  2020-12-31
Risk-free rate       0.00%       0.00%       0.00%       0.00%       0.00%       0.00%

Total Return         344.78%     443.79%     411.30%     167.23%     77.34%      79.25%
Daily Sharpe         1.22        1.28        1.23        0.76        0.64        0.69
Daily Sortino        1.99        2.09        2.09        1.21        1.01        1.06
CAGR                 34.85%      40.39%      38.67%      21.77%      12.16%      12.40%
Max Drawdown         -28.04%     -38.52%     -34.10%     -42.96%     -29.57%     -27.37%
Calmar Ratio         1.24        1.05        1.13        0.51        0.41        0.45

MTD                  3.90% 

C:\Users\chris\AppData\Local\conda\conda\envs\finrl\lib\site-packages\ffn\core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
C:\Users\chris\AppData\Local\conda\conda\envs\finrl\lib\site-packages\ffn\core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
C:\Users\chris\AppData\Local\conda\conda\envs\finrl\lib\site-packages\ffn\core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)


In [9]:
alpha, beta = alpha_beta(msft, benchmark)
print(beta)

1.0330866286823779


C:\Users\chris\AppData\Local\conda\conda\envs\finrl\lib\site-packages\empyrical\stats.py:1296: RuntimeWarning: overflow encountered in power
  out=out


In [10]:
returns = prices.pct_change()
 
# mean daily return and covariance of daily returns
mean_daily_returns = returns.mean()
cov_matrix = returns.cov()
 
# portfolio weights
weights = np.asarray([0.4,0.2,0.1,0.1,0.1,0.1])
 
portfolio_return = round(np.sum(mean_daily_returns * weights) * 252,2)
portfolio_std_dev = round(np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) * np.sqrt(252),2)

print("Expected annualised return: " + str(portfolio_return))
print("Volatility: " + str(portfolio_std_dev))

Expected annualised return: 0.3
Volatility: 0.23


In [11]:
# Expected returns and sample covariance
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

# Optimise portfolio for maximum Sharpe Ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)



OrderedDict([('msft', 0.16508), ('aapl', 0.46147), ('amzn', 0.37344), ('fb', 0.0), ('brkb', 0.0), ('jnj', 0.0)])
Expected annual return: 38.8%
Annual volatility: 26.3%
Sharpe Ratio: 1.40


(0.38827283271833685, 0.2633064620279706, 1.3986471501000073)

In [17]:
latest_prices = discrete_allocation.get_latest_prices(prices)
DAM = discrete_allocation.DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=100000)
allocation, leftover = DAM.lp_portfolio()
print('LP allocation:', allocation)
print("Funds remaining: ${:.2f}".format(leftover))
allocation, leftover = DAM.greedy_portfolio()
print('Greedy allocation:', allocation)
print("Funds remaining: ${:.2f}".format(leftover))

LP allocation: {'msft': 75, 'aapl': 348, 'amzn': 11}
Funds remaining: $1316.15
Greedy allocation: {'aapl': 347, 'amzn': 11, 'msft': 74}
Funds remaining: $1671.26


In [18]:
ef = EfficientFrontier(mu, S, weight_bounds=(-1, 1))
ef.efficient_return(target_return=0.2, market_neutral=True)

OrderedDict([('msft', 0.3268640397502882),
             ('aapl', 0.3609488909370011),
             ('amzn', 0.1794799869650703),
             ('fb', -0.2403210445459606),
             ('brkb', -0.4131380371437694),
             ('jnj', -0.2138338359626298)])